In [1]:
import pandas as pd
import numpy as np
import os
import math

In [5]:
!pip install -U sentence-transformers

In [2]:
import faiss
from sentence_transformers import SentenceTransformer, util

In [6]:
model = SentenceTransformer('paraphrase-distilroberta-base-v1') # Load the distil roberta model,whcih was trained on millions of data

A
 64%|██████▍   | 196M/306M [01:52<00:47, 2.34MB/s]
 64%|██████▍   | 196M/306M [01:52<00:43, 2.52MB/s]
 64%|██████▍   | 196M/306M [01:52<00:49, 2.22MB/s]
 64%|██████▍   | 196M/306M [01:52<00:55, 1.96MB/s]
 64%|██████▍   | 197M/306M [01:53<01:25, 1.27MB/s]
 65%|██████▍   | 197M/306M [01:53<00:52, 2.07MB/s]
 65%|██████▍   | 197M/306M [01:53<00:52, 2.04MB/s]
 65%|██████▍   | 198M/306M [01:53<00:53, 2.02MB/s]
 65%|██████▍   | 198M/306M [01:53<00:54, 1.99MB/s]
 65%|██████▍   | 198M/306M [01:53<00:55, 1.95MB/s]
 65%|██████▍   | 198M/306M [01:53<00:52, 2.03MB/s]
 65%|██████▌   | 199M/306M [01:53<00:50, 2.13MB/s]
 65%|██████▌   | 199M/306M [01:53<00:50, 2.12MB/s]
 65%|██████▌   | 199M/306M [01:54<00:52, 2.04MB/s]
 65%|██████▌   | 199M/306M [01:54<00:48, 2.18MB/s]
 65%|██████▌   | 200M/306M [01:54<00:46, 2.27MB/s]
 65%|██████▌   | 200M/306M [01:54<00:47, 2.21MB/s]
 65%|██████▌   | 200M/306M [01:54<00:49, 2.15MB/s]
 66%|██████▌   | 200M/306M [01:54<00:44, 2.34MB/s]
 66%|██████▌   | 201M/306M [0

In [7]:
movies = pd.read_csv('dataset/IMDb movies.csv')
movies.head()

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,tt0000009,Miss Jerry,Miss Jerry,1894,1894-10-09,Romance,45,USA,None,Alexander Black,...,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.,5.9,154,NaN,NaN,NaN,NaN,1.0,2.0
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,None,Charles Tait,...,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,589,$ 2250,NaN,NaN,NaN,7.0,7.0
2,tt0001892,Den sorte drøm,Den sorte drøm,1911,1911-08-19,Drama,53,"Germany, Denmark",NaN,Urban Gad,...,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.8,188,NaN,NaN,NaN,NaN,5.0,2.0
3,tt0002101,Cleopatra,Cleopatra,1912,1912-11-13,"Drama, History",100,USA,English,Charles L. Gaskill,...,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...,5.2,446,$ 45000,NaN,NaN,NaN,25.0,3.0
4,tt0002130,L'Inferno,L'Inferno,1911,1911-03-06,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",...,"Salvatore Papa, Arturo Pirovano, Giuseppe de L...",Loosely adapted from Dante's Divine Comedy and...,7.0,2237,NaN,NaN,NaN,NaN,31.0,14.0


In [8]:
imdb_movies = movies[pd.notnull(movies['description'])]
imdb_movies = imdb_movies.reset_index(drop=True)

In [9]:
imdb_movies.head()

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,tt0000009,Miss Jerry,Miss Jerry,1894,1894-10-09,Romance,45,USA,None,Alexander Black,...,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.,5.9,154,NaN,NaN,NaN,NaN,1.0,2.0
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,None,Charles Tait,...,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,589,$ 2250,NaN,NaN,NaN,7.0,7.0
2,tt0001892,Den sorte drøm,Den sorte drøm,1911,1911-08-19,Drama,53,"Germany, Denmark",NaN,Urban Gad,...,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.8,188,NaN,NaN,NaN,NaN,5.0,2.0
3,tt0002101,Cleopatra,Cleopatra,1912,1912-11-13,"Drama, History",100,USA,English,Charles L. Gaskill,...,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...,5.2,446,$ 45000,NaN,NaN,NaN,25.0,3.0
4,tt0002130,L'Inferno,L'Inferno,1911,1911-03-06,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",...,"Salvatore Papa, Arturo Pirovano, Giuseppe de L...",Loosely adapted from Dante's Divine Comedy and...,7.0,2237,NaN,NaN,NaN,NaN,31.0,14.0


In [10]:
'''
Sentence Transformers will be used to encode sentences into FAISS. To encode into FAISS, we need to have an unique ID (numeric) assigned to each sentence. For this we will use the index of the row of the movie description
'''
imdb_movies['id'] = imdb_movies.index

In [11]:
imdb_movies.head()

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics,id
0,tt0000009,Miss Jerry,Miss Jerry,1894,1894-10-09,Romance,45,USA,None,Alexander Black,...,The adventures of a female reporter in the 1890s.,5.9,154,NaN,NaN,NaN,NaN,1.0,2.0,0
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,None,Charles Tait,...,True story of notorious Australian outlaw Ned ...,6.1,589,$ 2250,NaN,NaN,NaN,7.0,7.0,1
2,tt0001892,Den sorte drøm,Den sorte drøm,1911,1911-08-19,Drama,53,"Germany, Denmark",NaN,Urban Gad,...,Two men of high rank are both wooing the beaut...,5.8,188,NaN,NaN,NaN,NaN,5.0,2.0,2
3,tt0002101,Cleopatra,Cleopatra,1912,1912-11-13,"Drama, History",100,USA,English,Charles L. Gaskill,...,The fabled queen of Egypt's affair with Roman ...,5.2,446,$ 45000,NaN,NaN,NaN,25.0,3.0,3
4,tt0002130,L'Inferno,L'Inferno,1911,1911-03-06,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",...,Loosely adapted from Dante's Divine Comedy and...,7.0,2237,NaN,NaN,NaN,NaN,31.0,14.0,4


In [12]:
sentences = imdb_movies['description'].tolist()
print(f'Number of sentences: {len(sentences)}')

Number of sentences: 83740


In [ ]:
embeddings = model.encode(sentences)
faiss.normalize_L2(embeddings)